# Zadanie 5 (22.12.2024)
Korzystając ze omawianego zbioru BBC, połącz klasę 1 i -1 w jedną i dokonaj klasyfikacji za pomocą metody SVM (jądro liniowe), porównując klasę subiektywną (1 i -1) z obiektywną (0). Badania przeprowadź dla w miarę podobnych liczb postów z każdej klasy (np. 500 i 500). Dla chętnych: sprawdź, czy zastosowanie transforami tf-idf doprowadzi do jakiejś zmiany w skuteczności klasyfikatora.

In [3]:
# import pandas as pd
# import numpy as np

# bbc = pd.read_csv("https://jsienkiewicz.pl/TEXT/lab/data_bbc.csv",  on_bad_lines='skip', encoding = "ISO-8859-1")
# bbc = bbc.sort_values(by = "emo")

# display(bbc.head())

,doc,text,emo
0,1,The world keeps quite as 21 synogogues are des...,-1
5422,5423,"Rest assured, Grant is the odd one out..",-1
5420,5421,Does anyone know how many muslims live in euro...,-1
5419,5420,"I don't have a problem with spreadsheets,...",-1
5418,5419,"Hi Evening News, Thank you for your considere...",-1


In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score

# Wczytanie danych
url = "https://jsienkiewicz.pl/TEXT/lab/data_bbc.csv"
data = pd.read_csv(url,  on_bad_lines='skip', encoding = "ISO-8859-1")

# Połączenie klas 1 i -1 w jedną klasę subiektywną (oznaczoną jako 1)
data['emo'] = data['emo'].apply(lambda x: 1 if x in [1, -1] else 0)

# Wyodrębnienie postów z klasy 0 i 1
class_0 = data[data['emo'] == 0]
class_1 = data[data['emo'] == 1]

# Zrównoważenie liczby postów w każdej klasie (po 500)
sample_size = 500
class_0_sample = class_0.sample(n=sample_size, random_state=42)
class_1_sample = class_1.sample(n=sample_size, random_state=42)

# Połączenie próbek w jeden zbiór danych
balanced_data = pd.concat([class_0_sample, class_1_sample])

# Podział na cechy (X) i etykiety (y)
X = balanced_data['text']
y = balanced_data['emo']

# Drop rows where the text is NaN
balanced_data = balanced_data.dropna(subset=['text'])

# Ensure all text is of type string
balanced_data['text'] = balanced_data['text'].astype(str)

# Clean the data first
balanced_data['text'] = balanced_data['text'].astype(str).fillna("")

# Re-split after cleaning
X = balanced_data['text']
y = balanced_data['emo']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Przekształcenie tekstu na wektory cech za pomocą CountVectorizer
vectorizer = CountVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# Inicjalizacja i trenowanie klasyfikatora SVM z jądrem liniowym
svm_classifier = SVC(kernel='linear', random_state=42)
svm_classifier.fit(X_train_vec, y_train)

# Predykcja na zbiorze testowym
y_pred = svm_classifier.predict(X_test_vec)

# Ocena modelu
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Dokładność modelu: {accuracy:.2f}")
print("Raport klasyfikacji:")
print(report)


Dokładność modelu: 0.62
Raport klasyfikacji:
              precision    recall  f1-score   support

           0       0.60      0.59      0.59        94
           1       0.64      0.66      0.65       106

    accuracy                           0.62       200
   macro avg       0.62      0.62      0.62       200
weighted avg       0.62      0.62      0.62       200

